In [168]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

# **Importación de los datos de properati y AR**

In [169]:
data = pd.read_csv("../data/properatti.csv", sep = ",", low_memory=False) 
#data.head(3)
data.shape

(121220, 26)

In [170]:
#import AR.csv trayendo algunas de las columnas que tienen info de ubicación
#columnas = ['num', 'place_name', 'place_name2','s20', 'Lat','Lon', 's', 'rnch', 'Pais','s0','s1','s2','s3','s4','s5','s6','s7','place_with_parent_names','fecha']
#dataAR = pd.read_csv("../data/AR.tsv", header=None, names=columnas,  usecols=[1,2,4,5,8,17], sep = "\t", low_memory=False) 

#elimino la cadena de caracteres America/ y separo la columna place_with_parent_names en columnas, 
#renombro las columnas resultantes para coincidir con las columnas de data
#dataAR["place_with_parent_names"] = dataAR["place_with_parent_names"].replace({'America/':''}, regex=True)
#dataAR[["country_name", "state_name"]] = dataAR["place_with_parent_names"].str.split("/", expand=True)
#dataAR["state_name"]=dataAR["state_name"].replace({'_':' '}, regex=True)
#dataAR.drop("place_with_parent_names", axis=1, inplace=True)
#dataAR.head(3)
#data.shape

# Sellecion solo de AMBA del dataset

In [206]:
#dejamos solo lo de AMBA
data = data[(data["state_name"]== 'Capital Federal') | (data["state_name"]== 'Bs.As. G.B.A. Zona Sur') | (data["state_name"]=='Bs.As. G.B.A. Zona Norte') | (data["state_name"]== 'Bs.As. G.B.A. Zona Oeste')]
print(data.shape)

(81150, 30)


# **Imputaciones**

## --completo valores de rooms con información del title, o properati_url o description

### Completar info Habitaciones


In [171]:
#Comprobamos el tipo y los valores.
print("tipo  : ", data.rooms.dtype)
print("nulos : ", data.rooms.isnull().sum())
data.rooms.unique()

tipo  :  float64
nulos :  73830


array([nan,  1.,  4.,  3.,  2.,  6.,  5., 10.,  7.,  9.,  8., 17., 22.,
       15., 12., 11., 14., 16., 20., 13., 25., 19., 30., 18., 32., 24.,
       31., 21., 29., 27., 23., 28.])

In [172]:
totalVentas = data.operation.notnull().sum()
totalSinCuartos = data.rooms.isnull().sum()
print('Total Registros \t\t\t', data.operation.notnull().sum() )
print('Total sin ambientes \t\t\t',data.rooms.isnull().sum() )
print('Porcentaje sin ambientes \t\t', round( (totalVentas- totalSinCuartos) / totalVentas * 100, 2), ' %' )


Total Registros 			 121220
Total sin ambientes 			 73830
Porcentaje sin ambientes 		 39.09  %


El dataset esta completo en un 39.09% para todas las propiedades, podemos hacer foco en los departamentos:

A continuación se propone recuperar los datos faltantantes desde las columnas de título y descripción de la publicación. Ponemos en mayúscula los títulos y descripciones y hacemos una muestra de los que tienen incompletos `rooms`.

In [173]:
data['title'] = data.title.str.upper()
data['description'] = data.description.str.upper()
data[['rooms', 'title', 'description']].sample(1)

,rooms,title,description
40992,6.0,CASA EN EL BARRIO BOCA RATON,CORREDOR RESPONSABLE: PEDRO J. MOLDES - CSI 62...


Reemplazamos los números escritos como palabras y todas la variantes de llamar a los monoambientes por '1 AMBIENTE'.

In [174]:
reemplazo_dic = {"UNO":"1", "DOS":"2", "TRES":"3", "CUATRO":"4", "CINCO":"5", "SEIS":"6", "SIETE":"7", "OCHO":"8",
             "NUEVE":"9", "DIEZ":"10", "MONOAMBIENTE":"1 AMBIENTE", "MONOAMB" : "1 AMBIENTE", "UN":"1", "AMBIENTE DIVISIBLE":"1 AMBIENTE",
             "MONO AMBIENTE": "1 AMBIENTE","DORMITORIOS": "AMBIENTE","DORMITORIO": "AMBIENTE","HABITACIONES": "AMBIENTE","HABITACION": "AMBIENTE"}
for key in reemplazo_dic.keys():
    data.description = data.description.str.replace(key, reemplazo_dic[key], regex=False)

for key in reemplazo_dic.keys():
    data.title = data.title.str.replace(key, reemplazo_dic[key], regex=False)


Recuperamos los ambientes desde título, descripción y actualizamos rooms.

In [175]:
controlRooms = data.loc[(data.rooms.isnull())]
controlRooms.rooms = controlRooms.description.str.extract("(\d+) AMB").astype(float)
data.update(controlRooms)

C:\Users\msaied\Anaconda3\envs\ds\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [176]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.title.str.extract("(\d+) AMB").astype(float)
data.update(controlRooms)

In [177]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.description.str.extract("(\d+) DORM").astype(float) + 1
data.update(controlRooms)

In [178]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.title.str.extract("(\d+) DORM").astype(float) + 1
data.update(controlRooms)

In [179]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.title.str.extract("(\d+)AMB").astype(float)
data.update(controlRooms)

In [180]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.description.str.extract("(\d+)AMB").astype(float)
data.update(controlRooms)

In [181]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.title.str.extract("(\d+)DORM").astype(float) + 1
data.update(controlRooms)

In [182]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.description.str.extract("(\d+) DORM").astype(float) + 1
data.update(controlRooms)

In [183]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.title.str.extract("(\d+)HABITACIO").astype(float) + 1
data.update(controlRooms)

In [184]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.title.str.extract("(\d+) HABITACIO").astype(float) + 1
data.update(controlRooms)

In [185]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.description.str.extract("(\d+)HABITACIO").astype(float) + 1
data.update(controlRooms)

In [186]:
controlRooms = data[(data.rooms.isnull()) ]
controlRooms.rooms = controlRooms.description.str.extract("(\d+) HABITACIO").astype(float) + 1
data.update(controlRooms)

In [187]:
controlRooms = data[(data.rooms.isnull()) ].sample(2)
controlRooms[['rooms', 'title', 'description']]

,rooms,title,description
73510,NaN,DEPARTAMENTO - SAAVEDRA,DEPARTAMENTO 50M² CON PUERTA DE GARAJE AUTOMÁT...
104540,NaN,DEPARTAMENTO - CONDOMINIOS INTERCONTINENTAL,"TIGRE, NORDELTA, CONDOMINIO INTERCONTINENAL - ..."


Volvemos a controlar luego del proceso el grado de llenado de la columna

In [188]:
totalVentas = data.operation.notnull().sum()
totalSinCuartos = data.rooms.isnull().sum()
porcentajeSinCuartos = round( (totalVentas- totalSinCuartos) / totalVentas * 100, 2)
print('Total Registros \t\t\t', totalVentas )
print('Total sin Ambientes \t\t\t', totalSinCuartos )
print('Porcentaje con ambientes \t\t', porcentajeSinCuartos, ' %' )

Total Registros 			 121220
Total sin Ambientes 			 16055
Porcentaje con ambientes 		 86.76  %


Luego del proceso se paso del 39,09 % de llenado para todas las propiedades del data set al 86,7 %. Veamos que pasa para cada tipo de propiedad

# **Limpieza de datos**

## --eliminar duplicados

In [189]:
#elimino duplicados 
data.drop_duplicates(keep="first")


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0.0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,2.0,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1.0,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,3.0,NaN,http://www.properati.com.ar/15bob_venta_depart...,VENTA DE DEPARTAMENTO EN DÉCIMO PISO AL FRENTE...,VENTA DEPTO 2 DORM. A ESTRENAR 7 E/ 36 Y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2.0,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,2.0,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3.0,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,3.0,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 AMBIENTES CON PATIO. HAY 3 DEPTOS EN LOTE...,PH 3 AMB. CFTE. RECICLADO,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4.0,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,1.0,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,121215.0,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,2.0,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.IMPECAB...,TORRE FORUM ALCORTA- IMPECABLE 3 AMBIENTES,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,121216.0,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,...,360.0,1383.333333,1383.333333,NaN,3.0,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,EXCELENTE E IMPECABLE CASA EN VENTA EN LAS LOM...,RUCA INMUEBLES | VENTA | LOMAS DE SAN ISIDRO |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,121217.0,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,39.0,2858.695652,3371.794872,NaN,1.0,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO 1 AMBIENTE A ESTRENAR BALCO...,VENTA DEPARTAMENTO 1 AMBIENTE A ESTRENAR BALCO...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,121218.0,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,...,48.0,1997.916667,1997.916667,NaN,2.0,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 AMB AL CONTRAFRENTE, LUMINOSO. EL DEPARTAME...",2 AMB. C/ DEP. DE SERVICIO AL CONTRAFRENTE| RE...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...


## --pasar todas las columnas a minusculas

In [190]:
data_lower = data.applymap(lambda x: x if np.isreal(x) else str(x).lower())
# comparo los tipos de datos antes y después de pasar a minúsculas:
#print(data_lower.dtypes == data.dtypes)
#print(data_lower.head(3))

## elimiar columnas no infiormativas

In [191]:
cols2keep = ['property_type', 'state_name','place_with_parent_names','price_aprox_usd', 'surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2', 'price_per_m2', 'rooms', 'description', 'title', 'properati_url']
data = data.loc[:, cols2keep]
#print(df_sub)

## limpieza de outliers

#### - - -Método 1 sin eliminación de registros: con el siguiente método reemplazo en el dataset original los outliers de más de 2 std de todas las columnas numericas por NaN, de una vez, sin tener en cuenta las particularidades de las columnas. Como resultado obtengo un dataset del mismo tamaño que el dataset original, con los outliers convertidos en NaN para no impactar en los datos de resumen.

In [192]:
#búsqueda y reemplazo de outliers (de más de 2 std, 95%) por NaN en las columnas numéricas, en un solo paso
df_sub = data.loc[:, 'price_usd_per_m2']
lim = np.abs((df_sub - df_sub.mean()) / df_sub.std(ddof=0)) < 2
data.loc[:, 'price_usd_per_m2'] = df_sub.where(lim, np.nan)
#data.head(3)
data.shape
#data.dropna()

(121220, 12)

# **Crear nuevas columnas con valor predictivo**

### Creamos columnas con info adicional que tendrán utilidad para predecir variación de precios.



In [193]:
#fracciono la columna properti_url para sacar la nube de palabras mas repetidas
import re
patron_url = re.compile(pattern="_", flags =re.IGNORECASE)
lista_url = data["properati_url"].apply(lambda x : patron_url.split(x))
serie_palabras = pd.Series(np.hstack(lista_url))
#serie_palabras.value_counts().head(20).plot(kind="bar")


In [194]:
serie_palabras.value_counts().head(50)

venta                            121086
departamento                      71019
garage                            56620
lavadero                          44034
balcon                            42286
casa                              40196
parrilla                          33381
piscina                           32146
luminoso                          31865
suite                             27101
placard                           26145
terraza                           25101
toilette                          22841
patio                             21168
vestidor                          18011
jardin                            17131
quincho                           12848
aire-acondicionado                12763
sum                               11665
dependencias                      11316
amenities                         10607
baulera                           10512
estrenar                          10379
vista                             10011
gimnasio                           9363


In [195]:
# a partir de la nube de palabras selecciono las que son buenos adicionales
adicionales = ["garage", "balcon", "parrilla", "piscina", "terraza", "patio", "jardin", "quincho", "sum", "amenities", "baulera", "gimnasio", "subte-linea-d", "subte-linea-b", "subte-linea-a", "subte-linea-h", "subte-linea-e" ]


In [196]:
#elimino el primer elemento de lista_url para no tener el elemento con el http: etc
for sublist in lista_url:
  del sublist[0]

In [197]:
lista_url

0         [venta, ph, mataderos, lavadero, patio, inmobi...
1         [venta, departamentos, la-plata, balcon, lavad...
2         [venta, departamentos, mataderos, lavadero, pl...
3                   [venta, ph, liniers, patio, g-goffredo]
4         [venta, departamentos, centro, cristina-pavone...
                                ...                        
121215    [venta, departamento, belgrano, balcon, suite,...
121216    [venta, casa, beccar, suite, hidromasaje, jard...
121217    [venta, departamento, villa-urquiza, holmberg,...
121218    [venta, departamento, plaza-colon, lavadero, l...
121219    [venta, departamento, capital-federal, baulera...
Name: properati_url, Length: 121220, dtype: object

In [198]:
#creo una función que compare la lista de palabras con la lista de listas
#y me da como resultado una lista de listas de palabras true/false segun coincida o no 
def buscador_palabras(quebuscar, dondebuscar):
  listadeextras = []
  for listas in dondebuscar:
    extras = []
    for palabra in quebuscar:
      if palabra in listas:
        extras.append(True)
      else:
        extras.append(False)
    listadeextras.append(extras)
  #print(listadeextras)
  return listadeextras     

In [199]:
#aplico la funcion a mi lista "adicionales" y "lista_url"
#chequeo que tenga la misma longitud de data
resultado = buscador_palabras(adicionales,lista_url)
len(resultado)

121220

In [200]:
#convierto resultado en dataframe, y renombro las columnas por la lista de palabras adicionales
df = pd.DataFrame(resultado)
df.columns = ["garage", "balcon", "parrilla", "piscina", "terraza", "patio", "jardin", "quincho", "s.u.m.", "amenities", "baulera", "gimnasio","subte-linea-d", "subte-linea-b", "subte-linea-a", "subte-linea-h", "subte-linea-e"]
df

,garage,balcon,parrilla,piscina,terraza,patio,jardin,quincho,s.u.m.,amenities,baulera,gimnasio,subte-linea-d,subte-linea-b,subte-linea-a,subte-linea-h,subte-linea-e
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False
121216,True,False,True,True,False,False,True,True,False,False,False,False,False,False,False,False,False
121217,True,True,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False
121218,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [201]:
#uno el dataframe original con el nuevo generado de true/false
data = pd.merge(data,df,left_index=True, right_index=True)
data.columns
#antes me puso los dos indices como resultado del merge, y tuve que sacar la primera columna
#data.drop(columns=data.columns[0], axis=1,inplace=True)
#data.columns

Index(['property_type', 'state_name', 'place_with_parent_names',
       'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
       'price_usd_per_m2', 'price_per_m2', 'rooms', 'description', 'title',
       'properati_url', 'garage', 'balcon', 'parrilla', 'piscina', 'terraza',
       'patio', 'jardin', 'quincho', 's.u.m.', 'amenities', 'baulera',
       'gimnasio', 'subte-linea-d', 'subte-linea-b', 'subte-linea-a',
       'subte-linea-h', 'subte-linea-e'],
      dtype='object')

In [202]:
#creo una columna que rellene el valor con la media del precio por m2 en este state_name
data["price_state"] = data.groupby('state_name')['price_usd_per_m2'].transform('mean') 
data.head(2)

,property_type,state_name,place_with_parent_names,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,description,...,s.u.m.,amenities,baulera,gimnasio,subte-linea-d,subte-linea-b,subte-linea-a,subte-linea-h,subte-linea-e,price_state
0,PH,Capital Federal,|Argentina|Capital Federal|Mataderos|,62000.0,55.0,40.0,1127.272727,1550.0,2.0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",...,False,False,False,False,False,False,False,False,False,2581.751290
1,apartment,Bs.As. G.B.A. Zona Sur,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,150000.0,NaN,NaN,NaN,NaN,3.0,VENTA DE DEPARTAMENTO EN DÉCIMO PISO AL FRENTE...,...,False,False,False,False,False,False,False,False,False,1478.390493


In [203]:
#creo una columna que rellene el valor con la media del m2 total en este state_name
data["price_state"] = data.groupby('state_name')['surface_total_in_m2'].transform('mean') 
data.head(2)

,property_type,state_name,place_with_parent_names,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,description,...,s.u.m.,amenities,baulera,gimnasio,subte-linea-d,subte-linea-b,subte-linea-a,subte-linea-h,subte-linea-e,price_state
0,PH,Capital Federal,|Argentina|Capital Federal|Mataderos|,62000.0,55.0,40.0,1127.272727,1550.0,2.0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",...,False,False,False,False,False,False,False,False,False,119.222453
1,apartment,Bs.As. G.B.A. Zona Sur,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,150000.0,NaN,NaN,NaN,NaN,3.0,VENTA DE DEPARTAMENTO EN DÉCIMO PISO AL FRENTE...,...,False,False,False,False,False,False,False,False,False,230.557603


# Inicio TP2

In [204]:
# para exportar resultados
data.to_csv(r'../data/properatti_tp2.csv', index = False, header=True)


In [205]:
#usar el nuevo dataset desde el archivo
data = pd.read_csv("../data/properatti_tp2.csv", sep = ",", low_memory=False) 
#data.head(3)
data.shape

(121220, 30)